##### Bibliotecas


In [1]:
import os
import numpy as np
import pandas as pd

##### Globais

In [6]:
diretorio        = 'datasets'
arquivo_extensao = '.csv'

##### Análise da Estrutura dos Dados (antes da limpeza)

Foram carregados bla bla bla

In [7]:
df_dados          = pd.DataFrame()    
df_estrutra_dados = pd.DataFrame()
lista_arquivos    = []

# Lista os arquivos do diretório
for arquivo in os.listdir(diretorio):
    if arquivo.endswith(arquivo_extensao):
       lista_arquivos.append(arquivo)

# Carrega o DataFrame 
for dataset in lista_arquivos:
    caminho_arquivo   = os.path.join(diretorio, dataset)
    df_dados          = pd.read_csv(caminho_arquivo, sep=';', low_memory=False)
    df_estrutra_dados = pd.concat([df_estrutra_dados, df_dados], ignore_index=True)

print("Dimensões da base")
print("-----------------")
print("")
print(f"Linhas: {len(df_estrutra_dados)}")
print(f"Colunas: {len(df_estrutra_dados.iloc[0])}")
    
print("")
print("") 
print("Estrutura")
print("---------")
print("")
print(df_estrutra_dados.info())

print("")
print("") 
print("Qtde de campos nulos")
print("--------------------")
print("")
print(df_estrutra_dados.isnull().sum())

# Visualização do DataFrame
print(display(df_estrutra_dados)) 

Tamanho da base
-----------------

Linhas: 10265783
Colunas: 16


Análise da estrutura
--------------------

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10265783 entries, 0 to 10265782
Data columns (total 16 columns):
 #   Column             Dtype 
---  ------             ----- 
 0   Regiao - Sigla     object
 1   Estado - Sigla     object
 2   Municipio          object
 3   Revenda            object
 4   CNPJ da Revenda    object
 5   Nome da Rua        object
 6   Numero Rua         object
 7   Complemento        object
 8   Bairro             object
 9   Cep                object
 10  Produto            object
 11  Data da Coleta     object
 12  Valor de Venda     object
 13  Valor de Compra    object
 14  Unidade de Medida  object
 15  Bandeira           object
dtypes: object(16)
memory usage: 1.2+ GB
None


Qtde de campos nulos
--------------------

Regiao - Sigla             0
Estado - Sigla             0
Municipio                  0
Revenda                    0
CNPJ da Rev

,Regiao - Sigla,Estado - Sigla,Municipio,Revenda,CNPJ da Revenda,Nome da Rua,Numero Rua,Complemento,Bairro,Cep,Produto,Data da Coleta,Valor de Venda,Valor de Compra,Unidade de Medida,Bandeira
0,SE,SP,GUARULHOS,AUTO POSTO SAKAMOTO LTDA,49.051.667/0001-02,RODOVIA PRESIDENTE DUTRA,S/N,"KM 210,5-SENT SP/RJ",BONSUCESSO,07178-580,GASOLINA,02/01/2013,"2,599","2,1955",R$ / litro,PETROBRAS DISTRIBUIDORA S.A.
1,SE,SP,GUARULHOS,AUTO POSTO SAKAMOTO LTDA,49.051.667/0001-02,RODOVIA PRESIDENTE DUTRA,S/N,"KM 210,5-SENT SP/RJ",BONSUCESSO,07178-580,ETANOL,02/01/2013,"1,859","1,5582",R$ / litro,PETROBRAS DISTRIBUIDORA S.A.
2,SE,SP,GUARULHOS,AUTO POSTO SAKAMOTO LTDA,49.051.667/0001-02,RODOVIA PRESIDENTE DUTRA,S/N,"KM 210,5-SENT SP/RJ",BONSUCESSO,07178-580,DIESEL,02/01/2013,"2,029",NaN,R$ / litro,PETROBRAS DISTRIBUIDORA S.A.
3,SE,SP,GUARULHOS,AUTO POSTO SAKAMOTO LTDA,49.051.667/0001-02,RODOVIA PRESIDENTE DUTRA,S/N,"KM 210,5-SENT SP/RJ",BONSUCESSO,07178-580,GNV,02/01/2013,"1,399",NaN,R$ / m³,PETROBRAS DISTRIBUIDORA S.A.
4,CO,DF,BRASILIA,GASOL COMBUSTÍVEIS AUTOMOTIVOS LTDA.,00.603.738/0001-43,"QI-QUADRA INTERNA 18 LOTES 47, 49, 51 E 53",S/N,NaN,TAGUATINGA,72315-000,GASOLINA,02/01/2013,"2,83",NaN,R$ / litro,PETROBRAS DISTRIBUIDORA S.A.
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10265778,S,RS,LAJEADO,SIM REDE DE POSTOS LTDA,07.473.735/0187-14,RODOVIA ERS-130,2285,KM 72.8,SANTO ANDRE,95912-000,GNV,26/06/2023,"5,79",NaN,R$ / m³,CHARRUA
10265779,S,RS,LAJEADO,SIM REDE DE POSTOS LTDA,07.473.735/0188-03,AVENIDA BENJAMIN CONSTANT,2324,NaN,FLORESTAL,95900-702,ETANOL,26/06/2023,"4,68",NaN,R$ / litro,CHARRUA
10265780,S,RS,LAJEADO,SIM REDE DE POSTOS LTDA,07.473.735/0188-03,AVENIDA BENJAMIN CONSTANT,2324,NaN,FLORESTAL,95900-702,DIESEL S10,26/06/2023,"4,68",NaN,R$ / litro,CHARRUA
10265781,S,RS,LAJEADO,SIM REDE DE POSTOS LTDA,07.473.735/0188-03,AVENIDA BENJAMIN CONSTANT,2324,NaN,FLORESTAL,95900-702,GASOLINA ADITIVADA,26/06/2023,"5,39",NaN,R$ / litro,CHARRUA


None


#### Carregamento dos datasets

In [53]:
def carregar_dados_estruturados(diretorio, lista_arquivos_csv):
    df_dados        = pd.DataFrame()
    df_dados_limpos = pd.DataFrame()
   
    for arquivo in lista_arquivos_csv:
        caminho_arquivo = os.path.join(diretorio, arquivo)
        df_dados        = pd.read_csv(caminho_arquivo,
                                      sep=';', thousands = '.', decimal = ',', 
                                      dtype = {'Regiao - Sigla': 'category',
                                               'Estado - Sigla': 'category',
                                               'Valor de Compra': np.float64})
        
        df_dados_limpos = pd.concat([df_dados_limpos, df_dados], ignore_index=True)
        df_dados_limpos = df_dados.drop(columns=['Nome da Rua',
                                                 'Numero Rua',
                                                 'Complemento',
                                                 'Bairro',
                                                 'Cep',
                                                 'Valor de Compra'])        
        
    return df_dados_limpos

#### Chamada para o carregamento dos Dados Estruturados (depois da limpeza)

In [ ]:
df_dados        = pd.DataFrame()
df_dados_limpos = pd.DataFrame()

for arquivo in lista_arquivos_csv:
    caminho_arquivo = os.path.join(diretorio, arquivo)
    df_dados        = pd.read_csv(caminho_arquivo,
                                    sep=';', thousands = '.', decimal = ',', 
                                    dtype = {'Regiao - Sigla': 'category',
                                            'Estado - Sigla': 'category',
                                            'Valor de Compra': np.float64})
    
    df_dados_limpos = pd.concat([df_dados_limpos, df_dados], ignore_index=True)
    df_dados_limpos = df_dados.drop(columns=['Nome da Rua',
                                                'Numero Rua',
                                                'Complemento',
                                                'Bairro',
                                                'Cep',
                                                'Valor de Compra'])        
    
return df_dados_limpos

df_dados.info()

In [55]:
lista_arquivos_csv = []
lista_arquivos_csv = listar_diretorio(diretorio, '.csv')

df_dados = pd.DataFrame()
df_dados = carregar_dados_estruturados(diretorio, lista_arquivos_csv)

df_dados.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 431576 entries, 0 to 431575
Data columns (total 10 columns):
 #   Column             Non-Null Count   Dtype   
---  ------             --------------   -----   
 0   Regiao - Sigla     431576 non-null  category
 1   Estado - Sigla     431576 non-null  category
 2   Municipio          431576 non-null  object  
 3   Revenda            431576 non-null  object  
 4   CNPJ da Revenda    431576 non-null  object  
 5   Produto            431576 non-null  object  
 6   Data da Coleta     431576 non-null  object  
 7   Valor de Venda     431576 non-null  float64 
 8   Unidade de Medida  431576 non-null  object  
 9   Bandeira           431576 non-null  object  
dtypes: category(2), float64(1), object(7)
memory usage: 27.2+ MB
None


#### Principal

In [ ]:
lista_arquivos_csv = []
lista_arquivos_csv = listar_diretorio(diretorio, '.csv')

df_dados = pd.DataFrame()
df_dados = carregar_dados_estruturados(diretorio, lista_arquivos_csv)

print(df_dados)

